In [1]:
import create_dict
import torch
import os
import json

# Clean Data

In [11]:
folder_name = '/data2'
#create_dict.clean_obj(folder_name)

# Generate Dataset

In [12]:
input_folder = '/data'
suffix = '_test' #suffix will append data_trainxxxx.json
max_elements_per_class = 5
create_dict.create_dictionary(input_folder, max_elements_per_class, suffix)

PywavefrontException: Unknown material: material_0_24

# Initialize Network

In [2]:
import SiameseNet

device = torch.device("cuda:0" if torch.cuda.torch.cuda.is_available() else "cpu")

batch_size = 2
net = SiameseNet.SiameseNet(batch_size)
suffix = '_test' # comment in if not coming from generating the dataset
path_to_params = "models/test.pt" # if file does not exist or is empty it starts from untrained and later saves to the file

# shift to GPU if available

net.to(device)
working_dir = os.getcwd()
data_dir_train = os.path.join(working_dir, 'data_train'+suffix+'.json')
data_dir_val = os.path.join(working_dir, 'data_val'+suffix+'.json')
class_dir = os.path.join(working_dir, 'class_dict'+suffix+'.json')

if os.path.isfile(path_to_params):
    if os.stat(path_to_params).st_size != 0:
        net.load_state_dict(torch.load(path_to_params))  #Loads pretrained net if file exists and if not empty
else:
    open(path_to_params, "x") #Creates parameter file if it does not exist

In [3]:
device

device(type='cpu')

# Train Network

In [4]:
#training parameters

writer_suffix = 'testing'
margin = 0.5
num_epochs = 1
print_batch = 1
lr = 0.0001



In [5]:
net = SiameseNet.train(net, num_epochs, margin, lr, print_batch, 
                       data_dir_train, data_dir_val, writer_suffix, path_to_params, working_dir)

63 train triplets generated -> return dict
Loaded 5032 word vectors.
Number of training triplets: 63
[1,     2] loss: 0.528
[1,     3] loss: 0.250
[1,     4] loss: 0.456
[1,     5] loss: 1.749
[1,     6] loss: 0.247
[1,     7] loss: 1.830
[1,     8] loss: 0.251
[1,     9] loss: 0.239
[1,    10] loss: 1.617
[1,    11] loss: 0.250
[1,    12] loss: 0.414
[1,    13] loss: 0.477
[1,    14] loss: 1.578
[1,    15] loss: 0.582
[1,    16] loss: 1.576
[1,    17] loss: 0.941
[1,    18] loss: 0.660
[1,    19] loss: 0.250
[1,    20] loss: 1.472
[1,    21] loss: 0.470
[1,    22] loss: 1.136
[1,    23] loss: 0.253
[1,    24] loss: 0.250
[1,    25] loss: 0.592
[1,    26] loss: 0.220
[1,    27] loss: 2.741
[1,    28] loss: 0.629
[1,    29] loss: 1.366
[1,    30] loss: 1.526
[1,    31] loss: 1.857
[1,    32] loss: 0.410
[1,    33] loss: 0.247
[1,    34] loss: 2.061
[1,    35] loss: 0.306
[1,    36] loss: 0.495
[1,    37] loss: 1.179
[1,    38] loss: 0.248
[1,    39] loss: 1.570
[1,    40] loss: 0.363
[1

# Validation

In [6]:
writer_suffix = 'Val'
margin = 0.5

y_true, y_pred, ids, shape, description = SiameseNet.retrieval(net, data_dir_val, working_dir)

8 val triplets generated
Loaded 5032 word vectors.
Number of validation triplets: 4
0 [7 0 6 1 4]
1 [7 0 6 1 4]
2 [7 0 6 1 4]
3 [7 6 0 1 4]
4 [7 0 6 1 4]
5 [7 0 6 1 4]
6 [7 0 6 1 4]
7 [7 0 6 1 4]


In [7]:
with open(data_dir_val, 'r') as fp:
    data_val = json.load(fp)

max_show = 3
for i, key in enumerate(ids):
    if (i>=max_show):
        break
    print("Sample: ID:", key, " Description:", data_val[key][1])
    print("1N: ID", ids[y_pred[i][0]]) #todo get picture
    print("2N: ID", ids[y_pred[i][1]]) #todo get picture

Sample: ID: cb38405e384c79c05e4e385b035691bb  Description: ['this', 'is', 'a', 'brown', 'tub']
1N: ID a8cfed0a1b9451c9d213467246722099
2N: ID cb38405e384c79c05e4e385b035691bb
Sample: ID: 555f9430c2ca273ccb2a965e75be701c  Description: ['this', 'is', 'a', 'brown', 'and', 'red', 'table']
1N: ID a8cfed0a1b9451c9d213467246722099
2N: ID cb38405e384c79c05e4e385b035691bb
Sample: ID: 2388c99892c2185268d1b9a1d97e2846  Description: ['this', 'is', 'up', 'lamp', 'around', 'a', 'blue', 'pole', 'with', 'no', 'lamp', 'shade']
1N: ID a8cfed0a1b9451c9d213467246722099
2N: ID cb38405e384c79c05e4e385b035691bb
